In [16]:
import numpy as np
import matplotlib.pyplot as plt
import wave
from scipy.linalg import toeplitz
from scipy.io.wavfile import write
import IPython.display as ipd

In [3]:
def read_audio(path):
    signal = wave.open(path, 'rb')
    signal_data = signal.readframes(-1)
    signal_data = np.frombuffer(signal_data, dtype='int16')
    return signal_data, signal.getframerate()

In [4]:
def prep(signal, P):
    N = len(signal)
    corr = np.correlate(signal, signal, mode='full')[N-1:]
    corr = corr/(N-P+1)
    corr_vec = corr[P:2*P]
    R = toeplitz(corr_vec)
    return R, corr_vec, corr

In [5]:
def train(Rx, corr_vec):
    wopt = np.linalg.inv(Rx) @ corr_vec
    return wopt

In [6]:
def process(signal, wopt):
    N = len(signal)
    y = np.zeros(N)
    for i in range(P, N):
        y[i] = wopt @ signal[i-P:i]
    return y

In [7]:
def export_audio(filename, signal, fs):
    audio = signal.astype('int16')
    write('./audio/'+filename+'.wav', fs, audio)
    print('Audio exported successfully!')

In [8]:
signal1, fs1 = read_audio('./audio/conv1.wav')
signal2, fs2 = read_audio('./audio/conv73.wav')

In [9]:
N = min(len(signal1), len(signal2))
signal1 = signal1[:N]
signal2 = signal2[:N]
signal = signal1 + signal2

In [10]:
frame_size = 512
hop_size = 256
P = 3

n_frames = int(np.floor((N/frame_size)*2-1))
whole_frames = int(np.floor(N/frame_size))

signal1 = signal1[:whole_frames*frame_size]
signal2 = signal2[:whole_frames*frame_size]
signal = signal[:whole_frames*frame_size]

signal1_frames = [signal1[i*hop_size:i*hop_size+frame_size] for i in range(n_frames-1)]
signal2_frames = [signal2[i*hop_size:i*hop_size+frame_size] for i in range(n_frames-1)]
signal_frames = [signal[i*hop_size:i*hop_size+frame_size] for i in range(n_frames-1)]

n_frames = n_frames - 1

wopt1_frames = np.zeros((n_frames, P))
wopt2_frames = np.zeros((n_frames, P))

In [11]:
for i in range(n_frames):
    frame1 = signal1_frames[i]
    frame2 = signal2_frames[i]
    R1, corr_vec1, corr1 = prep(frame1, P)
    R2, corr_vec2, corr2 = prep(frame2, P)
    wopt1_frames[i] = train(R1, corr_vec1)
    wopt2_frames[i] = train(R2, corr_vec2)

In [12]:
d1hat_frames = np.zeros((n_frames, frame_size))
d2hat_frames = np.zeros((n_frames, frame_size))

for i in range(n_frames):
    frame = signal_frames[i]
    d1hat_frames[i] = process(frame, wopt1_frames[i])
    d2hat_frames[i] = process(frame, wopt2_frames[i])

In [13]:
#CONCATENATE FRAMES
d1hat = np.zeros(N) 
d2hat = np.zeros(N)

for i in range(n_frames):
    d1hat[i*hop_size:i*hop_size+frame_size] = d1hat_frames[i]
    d2hat[i*hop_size:i*hop_size+frame_size] = d2hat_frames[i]

In [14]:
export_audio('d1hat_frames', d1hat, fs1)
export_audio('d2hat_frames', d2hat, fs2)

Audio exported successfully!
Audio exported successfully!


In [20]:
display(ipd.Audio(d1hat, rate=fs1))